In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from src.dataset import ProteinDataset
from src.utils import train_model, test_model
import torch
from src.model import ChemicalShiftsPredictor, ChemicalShiftsPredictorAttention
from src.utils import packed_padded_collate

from tqdm.notebook import tqdm
from catboost import CatBoostRegressor

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import h5py

/tmp/ipykernel_2333/1785675692.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
# Load and prepare data
csv_file = 'data/strict.csv'
prott5_file = 'data/embeddings/unfiltered_all_prott5.h5'
prott5_res_file = 'data/embeddings/unfiltered_all_prott5_res.h5'
prostt5_file = 'data/embeddings/prostt5.h5'
esm_file = 'data/embeddings/unfiltered_all_esm2_3b.h5'
esm_res_file = 'data/embeddings/unfiltered_all_esm2_3b_res.h5'
chemical_shifts_df = pd.read_csv(csv_file)
#chemical_shifts_df.describe()

In [3]:
test_ids = []
with open("pdb_matched/final_test_ids.txt", "r") as f:
    for line in f:
        test_ids.append(line.strip())

In [4]:
chemical_shifts_df = chemical_shifts_df[~chemical_shifts_df['ID'].isin(test_ids)]

In [5]:
chemical_shifts_df.dropna(inplace=True, subset=["H"])

In [6]:
# split dataset into train and test, based on unique protein IDs
ids = chemical_shifts_df['ID'].unique()
train_ids, test_ids = train_test_split(ids, test_size=0.2, random_state=42)
train_df = chemical_shifts_df[chemical_shifts_df['ID'].isin(train_ids)]
val_df = chemical_shifts_df[chemical_shifts_df['ID'].isin(test_ids)]

In [10]:
prostt5_embs = h5py.File(prostt5_file, "r")
prott5_embs = h5py.File(prott5_file, "r")

In [8]:
# get prosst5 embeddings for train and test
train_X = []
for row in tqdm(train_df.itertuples(), total=len(train_df)):
    protein_id = row.ID
    amino_acid_index = row.seq_index - 1  # Adjust for zero-based indexing
    amino_acid_prostt5_emb = prostt5_embs[protein_id][amino_acid_index]
    
    prot_emb = prott5_embs[protein_id]
    
    train_X.append(np.concatenate([amino_acid_prostt5_emb.flatten(), np.array(prot_emb).flatten()], axis=0))

  0%|          | 0/144661 [00:00<?, ?it/s]

In [9]:
val_X = []
for row in tqdm(val_df.itertuples(), total=len(val_df)):
    protein_id = row.ID
    amino_acid_index = row.seq_index - 1  # Adjust for zero-based indexing
    amino_acid_prostt5_emb = prostt5_embs[protein_id][amino_acid_index]
    
    prot_emb = prott5_embs[protein_id]
    
    val_X.append(np.concatenate([amino_acid_prostt5_emb.flatten(), np.array(prot_emb).flatten()], axis=0))


  0%|          | 0/36170 [00:00<?, ?it/s]

In [10]:
train_y = list(train_df['H'].values)
val_y = list(val_df['H'].values)

In [11]:
#model = CatBoostRegressor(iterations=2000, loss_function='RMSEWithUncertainty', posterior_sampling=True)
model = CatBoostRegressor(iterations=2000, task_type="GPU")
model.fit(train_X + val_X, train_y + val_y, verbose=True)

Learning rate set to 0.052759
0:	learn: 0.6883368	total: 69.2ms	remaining: 2m 18s
1:	learn: 0.6855053	total: 139ms	remaining: 2m 19s
2:	learn: 0.6829409	total: 207ms	remaining: 2m 17s
3:	learn: 0.6802690	total: 267ms	remaining: 2m 13s
4:	learn: 0.6780445	total: 328ms	remaining: 2m 11s
5:	learn: 0.6758982	total: 387ms	remaining: 2m 8s
6:	learn: 0.6729688	total: 441ms	remaining: 2m 5s
7:	learn: 0.6703826	total: 489ms	remaining: 2m 1s
8:	learn: 0.6686362	total: 544ms	remaining: 2m
9:	learn: 0.6669257	total: 601ms	remaining: 1m 59s
10:	learn: 0.6653715	total: 656ms	remaining: 1m 58s
11:	learn: 0.6638564	total: 711ms	remaining: 1m 57s
12:	learn: 0.6623399	total: 768ms	remaining: 1m 57s
13:	learn: 0.6609622	total: 832ms	remaining: 1m 58s
14:	learn: 0.6586148	total: 886ms	remaining: 1m 57s
15:	learn: 0.6573271	total: 941ms	remaining: 1m 56s
16:	learn: 0.6554259	total: 992ms	remaining: 1m 55s
17:	learn: 0.6535781	total: 1.04s	remaining: 1m 54s
18:	learn: 0.6517742	total: 1.09s	remaining: 1m 53

In [12]:
model.save_model("models/h_catboost_prostt5_5000iter.cbm")

In [3]:
# load catboost model
model = CatBoostRegressor()
model.load_model("models/h_catboost_prostt5_5000iter.cbm")

In [8]:
test_ids = []
with open("pdb_matched/final_test_ids.txt", "r") as f:
    for line in f:
        test_ids.append(line.strip())

In [11]:
chemical_shifts_df = pd.read_csv(csv_file)
# take test ids
chemical_shifts_df = chemical_shifts_df[chemical_shifts_df['ID'].isin(test_ids)]

#test on them
test_X = []
for row in tqdm(chemical_shifts_df.itertuples(), total=len(chemical_shifts_df)):
    protein_id = row.ID
    amino_acid_index = row.seq_index - 1  # Adjust for zero-based indexing
    amino_acid_prostt5_emb = prostt5_embs[protein_id][amino_acid_index]
    
    prot_emb = prott5_embs[protein_id]
    
    test_X.append(np.concatenate([amino_acid_prostt5_emb.flatten(), np.array(prot_emb).flatten()], axis=0))

test_y = list(chemical_shifts_df['H'].values)

preds = model.predict(test_X)

  0%|          | 0/12892 [00:00<?, ?it/s]

In [12]:
# select N_our, H_our, ID, entryID, seq_index, seq and save to csv
df = pd.DataFrame({'H_cat': preds, 'ID': chemical_shifts_df['ID'], 'entryID': chemical_shifts_df['entryID'], 'seq_index': chemical_shifts_df['seq_index'], 'seq': chemical_shifts_df['seq']})
df.to_csv("test_h_catboost_prostt5.csv", index=False)